## IR Assignment Group-082

## Group members

<style>
table {
  font-family: arial, sans-serif;
  border-collapse: collapse;
  width: 100%;
}

td, th {
  border: 1px solid #dddddd;
  text-align: left;
  padding: 8px;
}

tr:nth-child(even) {
  background-color: #dddddd;
}
</style>

<table>
  <tr>
    <th>Full Name</th>
    <th>BITS ID</th>
  </tr>
  <tr>
    <td>T V Sriharini</td>
    <td>2020fc04020@wilp.bits-pilani.ac.in</td>
  </tr>
  <tr>
    <td>Subodh Kant Mishra</td>
    <td>2020fc04064@wilp.bits-pilani.ac.in</td>
  </tr>
  <tr>
    <td>Mriganka Shekhar Gayen</td>
    <td>2020fc04069@wilp.bits-pilani.ac.in</td>
  </tr>
</table>

Given a Dataset collected from the Centre for Inventions and Scientific Information ("CISI") and consists of text data about 1,460 documents and 112 associated queries. Its purpose is to be used to build models of information retrieval where a given query will return a list of document IDs relevant to the query. The file "CISI.REL" contains the correct list (ie. "gold standard" or "ground proof") of query-document matching and your model can be compared against this "gold standard" to see how it has performed.

### 1. Load the data set and import necessary libraries (1 Marks)
We import various packages used for text and general data processing
* Pandas, Numpy, Skleaarn - For Data Processing
* NLTK - Natural Language Tool Kit for Natural Language Processing
* OS, functools - General Packages for Processing files


In [248]:
## Library imports
import numpy as np 
import pandas as pd
from rank_bm25 import BM25Okapi

from sklearn.feature_extraction.text import CountVectorizer
#we will use Scikit-Learn's TfidfVectorizer, which operates similarly to CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#implementation using scikit-learn
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

import os, glob, re, sys, random, unicodedata, collections
from tqdm import tqdm
from functools import reduce
import nltk

from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.tokenize import sent_tokenize , word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kants\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kants\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### 1.1 Importing the files - All FIle

We are importing the CISI.ALL file where we perform text processing such as removing newline characters, and '.' characters,
and split the data into different variables.

In [249]:
def extract_text(path,filetype="ALL"):
    """
    Extracts text from a given file path.
    """

    qry_id=0
    doc_id=0
    doc_text = ""
    rel_set = {}
    doc_set = {}
    qry_set = {}
    lines = ""
    with open(path, 'r', encoding='utf-8') as f:
        if filetype == "ALL" or filetype == "QRY":
            for l in f.readlines():
                lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
            lines = lines.lstrip("\n").split("\n")
        elif filetype == "REL":
            for l in f.readlines():
                qry_id = int(l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0]) -1
                doc_id = int(l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1])-1
                if qry_id in rel_set:
                    rel_set[qry_id].append(doc_id)
                else:
                    rel_set[qry_id] = []
                    rel_set[qry_id].append(doc_id)
            return rel_set
        else:
            print("Invalid file type")
            return lines
    if filetype == "ALL": 
        for l in lines:
            if l.startswith(".I"):
                doc_id = int(l.split(" ")[1].strip())-1
            elif l.startswith(".X"):
                doc_set[doc_id] = doc_text.lstrip(" ")
                doc_id = ""
                doc_text = ""
            else:
                doc_text += l.strip()[3:] + " " # The first 3 characters of a line can be ignored.
        return doc_set
    elif filetype == "QRY":
        qry_id = ""
        for l in lines:
            if l.startswith(".I"):
                qry_id = int(l.split(" ")[1].strip()) -1
            elif l.startswith(".W"):
                qry_set[qry_id] = l.strip()[3:]
                qry_id = ""
        return qry_set

In [250]:
### Processing DOCUMENTS
BASE_FOLDER = "D://Mtech//Sem3//IR//Assignment//archive//"
doc_set = extract_text(f'{BASE_FOLDER}CISI.ALL',filetype="ALL")
print(f'Read {len(doc_set)} documents')

Read 1460 documents


### 1.2 Importing the files - Queries FIle

We are importing the CISI.QRY file where we perform text processing such as removing newline characters, and '.' characters,
and split the data into different variables.

In [251]:
qry_set = extract_text(f'{BASE_FOLDER}CISI.QRY',filetype="QRY")
print(f'Read {len(qry_set)} documents')

Read 112 documents


### 1.3 Importing the files - Rel FIle

We are importing the CISI.QREL file where we perform text processing such as removing newline characters, and '.' characters,
and split the data into different variables.

In [252]:
rel_set = extract_text(f'{BASE_FOLDER}CISI.REL',filetype="REL")
print(f'Read {len(rel_set)} documents')

Read 76 documents


### 2.1 Extract features from text files. Each unique word in the document can be considered as a feature (1 Mark)

We check the General statistics of the file provided

In [253]:
## Here we check some statistics and info of CISI dataset

print(f'Read {len(doc_set)} documents, {len(qry_set)} queries and {len(rel_set)} mappings from CISI dataset')

number_of_rel_docs = [len(value) for key, value in rel_set.items()]
print('Average %.2f and %d min number of relevant documents by query ' % 
      (np.mean(number_of_rel_docs), np.min(number_of_rel_docs)))

print('Queries without relevant documents: ', 
      np.setdiff1d(list(qry_set.keys()),list(rel_set.keys())))

Read 1460 documents, 112 queries and 76 mappings from CISI dataset
Average 40.97 and 1 min number of relevant documents by query 
Queries without relevant documents:  [ 35  37  39  46  47  50  52  58  59  62  63  67  69  71  72  73  74  76
  77  79  82  84  85  86  87  88  90  92  93 102 104 105 106 107 109 111]


#### Checking the sample of relevant document from query and its content

In [254]:
random.seed(42)
idx = random.sample(rel_set.keys(),1)[0]

print('Query ID %s ==>' % idx, qry_set[idx])
rel_docs = rel_set[idx]
print('Documents relevants to Query ID %s' % idx, rel_docs)
sample_document_idx = random.sample(rel_docs,1)[0]
print('Document ID %s ==>' % sample_document_idx, doc_set[sample_document_idx])

Query ID 14 ==> How much do information retrieval and dissemination systems, as well as automated libraries, cost? Are they worth it to the researcher and to industry?
Documents relevants to Query ID 14 [17, 26, 35, 48, 55, 58, 66, 73, 82, 125, 157, 163, 166, 191, 213, 221, 222, 249, 280, 291, 294, 298, 306, 330, 335, 337, 347, 364, 365, 366, 367, 371, 380, 445, 457, 464, 465, 481, 489, 490, 494, 496, 506, 519, 527, 590, 593, 622, 628, 638, 689, 719, 722, 723, 726, 727, 730, 778, 821, 833, 838, 847, 848, 864, 871, 896, 1099, 1160, 1247, 1304, 1352, 1357, 1362, 1365, 1367, 1370, 1371, 1373, 1374, 1375, 1376, 1409]
Document ID 48 ==> Adaptive Information Dissemination Sage, C.R. Anderson, R.R. Fitzwater, D.R. Computer dissemination of information offers significant advantages over manual dissemination because the computer can use strategies that are impractical and in some cases impossible for a human.. This paper describes the Ames Laboratory Selective Dissemination of Information syste

## Deriving Feature using Count Vectorizer Method

In [255]:
# count term frequency using CountVectorizer from scikit-learn
## limiting number of words just for illustrating the concept
STOP_WORDS = set(stopwords.words('english'))
vec = CountVectorizer(stop_words=STOP_WORDS) 
X = vec.fit_transform(doc_set.values())
print('Non Zeros doc matrix',vec.transform(doc_set.values()).count_nonzero())
df = pd.DataFrame(X.todense(), columns=vec.get_feature_names())
df
#word_tokenize(str(preprocess(doc_set[0])))

Non Zeros doc matrix 84538


,00,000,029,044,054,071,071182,073,077,084,...,zipf,zipfian,zipperer,zone,zones,zoology,zuckerman,zunde,zvezhinskii,zyabrev
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1456,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1457,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1458,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [256]:
processed_text = []
processed_set = {}
for key in doc_set:
    processed_text.append(word_tokenize(str(preprocess(doc_set[key]))))
    processed_set[key] = word_tokenize(str(preprocess(doc_set[key])))
print(len(processed_set))

1460


In [257]:
len(processed_set)

1460

In [258]:
from collections import Counter

series = pd.Series(doc_set)
bag_of_words = (
    series.
    str.lower().                  # convert all letters to lowercase
    str.replace("[^\w\s]", " ").  # replace non-alphanumeric characters by whitespace
    str.split()                   # split on whitespace
).apply(Counter)
tf = pd.DataFrame(list(bag_of_words)).fillna(0)

df = (tf > 0).sum(axis=0)
# Get IDFs
idf = np.log(len(tf) / df)
# Calculate TF-IDFs
tf_idf = tf * idf
tf_idf

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


,18,editions,of,the,dewey,decimal,classifications,comaromi,j,p,...,clause,subsidization,morn,religiously,prompts,soundness,supposition,poluskin,abstraction,certificates
0,5.088967,10.680563,0.086838,0.144880,14.163727,9.027206,4.241669,7.286192,1.809728,2.797555,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.136459,0.101416,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.099243,0.086928,0.000000,0.000000,0.000000,0.000000,0.000000,2.797555,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.099243,0.130392,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.099243,0.130392,0.000000,0.000000,0.000000,0.000000,0.000000,2.797555,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.000000,0.000000,0.062027,0.202832,0.000000,0.000000,0.000000,0.000000,1.809728,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1456,0.000000,0.000000,0.136459,0.260784,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1457,0.000000,0.000000,0.099243,0.173856,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,7.286192,7.286192,7.286192,7.286192,7.286192,7.286192,7.286192,0.000000,0.000000,0.000000
1458,0.000000,0.000000,0.074432,0.086928,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.286192,7.286192,0.000000


In [259]:
#we will use Scikit-Learn's TfidfVectorizer, which operates similarly to CountVectorizer
vec = TfidfVectorizer(norm=None) # Do not normalize.
vec.fit(series) # This determines the vocabulary.
tf_idf_sparse = vec.transform(series)
tf_idf_sparse

<1460x11150 sparse matrix of type '<class 'numpy.float64'>'
	with 114611 stored elements in Compressed Sparse Row format>

### Calculating Cosine Similarity

In [260]:
cosine_similarity(tf_idf_sparse)
cosine_similarity(tf_idf_sparse)[0]

array([1.        , 0.10351387, 0.07485884, ..., 0.10059578, 0.08325116,
       0.09139324])

In [261]:
WORD_MIN_LENGTH = 2 ## we'll drop all tokens with less than this size

def convert_lower_case(data):
    return np.char.lower(data)

def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

def strip_accents(text):
    """Strip accents and punctuation from text. 
    For instance: strip_accents("João e Maria, não entrem!") 
    will return "Joao e Maria  nao entrem "

    Parameters:
    text (str): Input text

    Returns:
    str: text without accents and punctuation

   """    
    nfkd = unicodedata.normalize('NFKD', text)
    newText = u"".join([c for c in nfkd if not unicodedata.combining(c)])
    return re.sub('[^a-zA-Z0-9 \\\']', ' ', newText)

def tokenize_text(text):
    """Make all necessary preprocessing of text: strip accents and punctuation, 
    remove \n, tokenize our text, convert to lower case, remove stop words and 
    words with less than 2 chars.

    Parameters:
    text (str): Input text

    Returns:
    str: cleaned tokenized text

   """        
    text = strip_accents(text)
    text = re.sub(re.compile('\n'),' ',text)
    words = word_tokenize(text)
    words = [word.lower() for word in words]
    words = [word for word in words if word not in STOP_WORDS and len(word) >= WORD_MIN_LENGTH]
    return words
    
def inverted_index(words):
    """Create a inverted index of words (tokens or terms) from a list of terms

    Parameters:
    words (list of str): tokenized document text

    Returns:
    Inverted index of document (dict)

   """       
    inverted = {}
    for index, word in enumerate(words):
        locations = inverted.setdefault(word, [])
        locations.append(index)
    return inverted

def inverted_index_add(inverted, doc_id, doc_index):
    """Insert document id into Inverted Index

    Parameters:
    inverted (dict): Inverted Index
    doc_id (int): Id of document been added
    doc_index (dict): Inverted Index of a specific document.

    Returns:
    Inverted index of document (dict)

   """        
    for word in doc_index.keys():
        locations = doc_index[word]
        indices = inverted.setdefault(word, {})
        indices[doc_id] = locations
    return inverted

## Using AND as logical operator
def boolean_search(inverted, file_names, query):
    """Run a boolean search with AND operator between terms over 
    the inverted index.

    Parameters:
    inverted (dict): Inverted Index
    file_names (list): List with names of files (books)
    query (txt): Query text

    Returns:
    Names of books that matchs the query.

   """      
    # preprocess the user query using same function used to build Inverted Index
    words = [word for _, word in enumerate(tokenize_text(query)) if word in inverted]
    # list with a disctinct document match for each term from query
    results = [set(inverted[word].keys()) for word in words]
    # AND operator. Replace & for | to modify to OR behavior.
    docs = reduce(lambda x, y: x & y, results) if results else []
    return ([file_names[doc] for doc in docs])

def ranked_search(k, tf_idf_index, file_names, query):
    """Run ranked query search using tf-idf model.

    Parameters:
    k (int): number of results to return
    tf_idf_index (dict): Data Structure storing Tf-Idf weights to each 
                        pair of (term,doc_id) 
    file_names (list): List with names of files (books)
    query (txt): Query text

    Returns:
    Top-k names of books that matchs the query.

   """   
    tokens = tokenize_text(query)
    query_weights = {}
    for doc_id, token in tf_idf:
        if token in tokens:
            query_weights[doc_id] = query_weights.get(doc_id, 0) + tf_idf_index[doc_id, token]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)
    results = []
    for i in query_weights[:k]:
        results.append(int(file_names[i[0]]))
    results.sort()
    return results

## TF-IDF Calculation

In [262]:
inverted_doc_indexes = {}
files_with_index = []
files_with_tokens = {}
doc_id=0
for fname in tqdm(doc_set):
    text = doc_set[fname]

    #Clean and Tokenize text of each document
    words = tokenize_text(text)
    #Store tokens
    files_with_tokens[doc_id] = words

    doc_index = inverted_index(words)
    inverted_index_add(inverted_doc_indexes, doc_id, doc_index)
    files_with_index.append(fname)
    doc_id = doc_id+1

100%|██████████| 1460/1460 [00:01<00:00, 981.77it/s] 


#### Searching from keyword using inverted index set. Also using boolean search to derive the document id and its content

In [263]:
## Check presence of zyabrev token into 1105 doocument:
zdocs = inverted_doc_indexes['centers']
zdocs_id = 0
for idx in zdocs.keys():
    zdocs_id = files_with_index[idx]
print(zdocs_id,'\n----\n' ,doc_set[zdocs_id],'\n----')
# Passage from "1105 doc"
print(boolean_search(inverted_doc_indexes, files_with_index, "survey, questionnaire; electronics, system"))

1395 
----
 Survey of Commercially Available Computer-Readable Bibliographic Data Bases Schneider, John H. Gechman, Marvin Furth, Stephen E. This document contains the results of a survey of 94 US organizations, and 36 organizations in other countries that were thought to prepare machine-readable data bases.. Of those surveyed, 55 organizations (40 in U.S., 15 in other countries) provided completed camera-ready forms describing bibliographic information about published literature.. The following types of data were requested for each data base:  Name, frequency of issue, and time span covered by the data base; Name of organizations and individuals who can provide information on the data base; Subject matter and scope of data on the tape; Source of information in the data base (journal articles, reports, patents, monographs, etc.); Method(s) used for indexing or other types of subject analysis; Special data elements; Tape specifications (density, tracks, labels, etc.); Availability of pr

In [264]:
## Number of documents each term occurs
DF = {}
for word in inverted_doc_indexes.keys():
    DF[word] = len ([doc for doc in inverted_doc_indexes[word]])

total_vocab_size = len(DF)
print(total_vocab_size)

tf_idf = {} # Our data structure to store Tf-Idf weights
N = len(files_with_tokens)
for doc_id, tokens in tqdm(files_with_tokens.items()):
    counter = Counter(tokens)
    words_count = len(tokens)
    
    for token in np.unique(tokens):
        # Calculate Tf
        tf = counter[token] # Counter returns a tuple with each terms counts
        tf = 1+np.log(tf)
        # Calculate Idf
        if token in DF:
            df = DF[token]
        else:
            df = 0
        idf = np.log((N+1)/(df+1))
        # Calculate Tf-idf        
        tf_idf[doc_id, token] = tf*idf
len(tf_idf)

11092


100%|██████████| 1460/1460 [00:00<00:00, 2823.80it/s]


84840

In [265]:
print(qry_set[1],'\n--- Searched Doc Ids ---\n',ranked_search(20, tf_idf, files_with_index, qry_set[1]),'----------\n',rel_set[1])


How can actually pertinent data, as opposed to references or entire articles themselves, be retrieved automatically in response to information requests? 
--- Searched Doc Ids ---
 [64, 165, 201, 209, 380, 522, 602, 635, 787, 789, 805, 809, 1053, 1054, 1090, 1154, 1157, 1275, 1398, 1417] ----------
 [28, 67, 196, 212, 213, 308, 318, 323, 428, 498, 635, 668, 669, 673, 689, 691, 694, 699, 703, 708, 719, 730, 732, 737, 739, 1135]


### Pseudo Relevance Feedback Implementation

In [266]:
idx = 10
query = qry_set[idx] #get query text
rel_docs = rel_set[idx] #get relevant documents

# Index all documents using BM25
corpus = list(doc_set.values())
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

# Process query and get scores for each indexed document using BM25
tokenized_query = query.split(" ")
print('Query ==> ', query, '\nRelevant documents IDs: ==> ', rel_docs)
scores = bm25.get_scores(tokenized_query)
print('bm25 scores -----> ',scores)

Query ==>  What is the need for information consolidation, evaluation, and retrieval in scientific research? 
Relevant documents IDs: ==>  [1, 14, 31, 32, 36, 40, 62, 66, 86, 89, 94, 109, 112, 115, 120, 152, 159, 162, 164, 165, 168, 171, 198, 201, 214, 218, 222, 242, 251, 253, 265, 308, 313, 326, 344, 354, 355, 359, 370, 371, 372, 380, 385, 390, 394, 397, 414, 438, 439, 452, 455, 512, 532, 543, 546, 559, 601, 604, 655, 656, 665, 713, 715, 717, 718, 722, 727, 730, 732, 736, 737, 738, 762, 770, 771, 787, 788, 820, 827, 836, 838, 844, 898, 951, 952, 964, 966, 1079, 1080, 1088, 1093, 1094, 1095, 1097, 1098, 1101, 1109, 1112, 1113, 1120, 1121, 1127, 1150, 1154, 1155, 1157, 1159, 1222, 1227, 1234, 1240, 1253, 1255, 1256, 1263, 1267, 1274, 1283, 1294, 1297, 1302, 1308, 1318, 1372, 1403, 1417, 1445]
bm25 scores ----->  [11.35530779 12.61482487 14.49104357 ... 11.2859353   9.49749892
 16.78178902]


In [267]:
##Source: https://gist.github.com/bwhite/3726239
def mean_reciprocal_rank(bool_results, k=10):
    """Score is reciprocal of the rank of the first relevant item
    First element is 'rank 1'.  Relevance is binary (nonzero is relevant).
    Example from http://en.wikipedia.org/wiki/Mean_reciprocal_rank
    >>> rs = [[0, 0, 1], [0, 1, 0], [1, 0, 0]]
    >>> mean_reciprocal_rank(rs)
    0.61111111111111105

    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Mean reciprocal rank
    """
    bool_results = (np.atleast_1d(r[:k]).nonzero()[0] for r in bool_results)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in bool_results])

mean_reciprocal_rank([[0, 0, 1], [0, 1, 0], [1, 0, 0]])


0.611111111111111

In [276]:
## Argsort gives the indexes of values in increasing order, so we input with the negative values of scores
most_relevant_documents = np.argsort(-scores)

print(most_relevant_documents[:100]) # printing first 100 most relevant results

## Mask relevant documents with 0's and 1's according to query <-> document annotation
masked_relevance_results = np.zeros(most_relevant_documents.shape)
masked_relevance_results[rel_docs] = 1
sorted_masked_relevance_results = np.take(masked_relevance_results, most_relevant_documents)

print(sorted_masked_relevance_results[:100]) #printing first 100 results: 1 is relevant 0 isn't

# Calculate MRR@10
print(mean_reciprocal_rank([sorted_masked_relevance_results]))

[ 460  134  574  458  387  133  593  595  543  610  397   59  559  894
  480   23  363  898  629  444 1143  524 1050  982  500 1357  636  250
  165  584  124  382  706  614  455  647  266  334  946 1200  483 1088
 1119 1079  975 1222  685  635  326 1406  370   65  452  546  895    5
 1447 1174  663  175  937 1163 1348 1161 1179  173  128  495   70  239
 1122 1093  947 1459  485  715  705  964  496  537  797  340  167  457
  113  754 1157 1308 1123  313  514 1185  174  812    7  322 1166 1125
  141  258]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0.
 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
0.1111111111111111


In [277]:
def results_from_query(qry_id, bm25):
    """Return an ordered array of relevant documents returned by query_id

    Args:
        qry_id (int): id of query on dataset
        bm25 (object): indexed corpus

    Returns:
        boolean sorted relevance array of documents
    """    
    query = qry_set[qry_id]
    rel_docs = []
    if qry_id in rel_set:
        rel_docs = rel_set[qry_id]
    tokenized_query = query.split(" ")
    scores = bm25.get_scores(tokenized_query)
    most_relevant_documents = np.argsort(-scores)
    masked_relevance_results = np.zeros(most_relevant_documents.shape)
  
    masked_relevance_results[rel_docs] = 1
    sorted_masked_relevance_results = np.take(masked_relevance_results, most_relevant_documents)
    
    return sorted_masked_relevance_results


results = [results_from_query(qry_id, bm25) for qry_id in list(qry_set.keys())]
print('MRR@10 %.4f' % mean_reciprocal_rank(results))

MRR@10 0.1483


In [278]:
# Instaciate objects from NLTK
stemmer = nltk.stem.PorterStemmer()
stop_words = nltk.corpus.stopwords.words('english')

In [279]:
def preprocess_string(txt, remove_stop=True, do_stem=True, to_lower=True):
    """
    Return a preprocessed tokenized text.
    
    Args:
        txt (str): original text to process
        remove_stop (boolean): to remove or not stop words (common words)
        do_stem (boolean): to do or not stemming (suffixes and prefixes removal)
        to_lower (boolean): remove or not capital letters.
        
    Returns:
        Return a preprocessed tokenized text.
    """    
    if to_lower:
        txt = txt.lower()
    tokens = nltk.tokenize.word_tokenize(txt)
    
    if remove_stop:
        tokens = [tk for tk in tokens if tk not in stop_words]
    if do_stem:
        tokens = [stemmer.stem(tk) for tk in tokens]
    return tokens

In [280]:
corpus = list(doc_set.values())
# You may experiment with this trying to improve MRR@10
remove_stop = True
do_stem = True
to_lower = True

tokenized_corpus = [preprocess_string(doc, remove_stop, do_stem, to_lower) for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

def results_from_query_new(qry_id, bm25):
    query = qry_set[qry_id]
    rel_docs = []
    if qry_id in rel_set:
        rel_docs = rel_set[qry_id]
    tokenized_query = preprocess_string(query, remove_stop, do_stem, to_lower)
    scores = bm25.get_scores(tokenized_query)
    most_relevant_documents = np.argsort(-scores)
    masked_relevance_results = np.zeros(most_relevant_documents.shape)
corpus = list(doc_set.values())
# You may experiment with this trying to improve MRR@10
remove_stop = True
do_stem = True
to_lower = True

tokenized_corpus = [preprocess_string(doc, remove_stop, do_stem, to_lower) for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

def results_from_query_new(qry_id, bm25):
    query = qry_set[qry_id]
    rel_docs = []
    if qry_id in rel_set:
        rel_docs = rel_set[qry_id]
    tokenized_query = preprocess_string(query, remove_stop, do_stem, to_lower)
    scores = bm25.get_scores(tokenized_query)
    most_relevant_documents = np.argsort(-scores)
    masked_relevance_results = np.zeros(most_relevant_documents.shape)
    masked_relevance_results[rel_docs] = 1
    sorted_masked_relevance_results = np.take(masked_relevance_results, most_relevant_documents)
    return sorted_masked_relevance_results


results = [results_from_query_new(qry_id, bm25) for qry_id in list(qry_set.keys())]
print('MRR@10 %.4f' % mean_reciprocal_rank(results))

MRR@10 0.4187
